In [ ]:
"""
author: William Darko (repurposed from original author Francois Chollet)
date: July, 2021
description: Regression problem using keras provided Boston housing dataset; example taken, an repurposed from 'Deep Learning with Python' (Manning publication, 1st ed) book by Francois Chollet
"""

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
from keras.datasets import boston_housing


In [ ]:
# loading, and inspecting the data

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print("Training data shape: ", train_data.shape)
print("Training targets shape: ", train_targets.shape)
print(train_data[:10])

# train_data.shape[0] corresponds to number of samples, in this case 404
# train_data.shape[1] corresponds to number of features per sample, in this case 13
# each sample represents a particular suburb in the Boston area, with 13 features such as crime rate, etc.
# targets represent median values of homes in thousands of dollars per sample

In [ ]:
# normalising the data (feature-wise normalisaiton of data)
# because features values take wildy different ranges, this would make learning more difficult, thus feature normalisation is required to
# centre values around 0, and have unit standard deviations. This requires taking the mean of each feature column and for each feature subtracting the mean, and dividing by the standard deviation. This method is also recognised as Standardisation, or Z-score

feature_mean = train_data.mean(axis=0)
# pass axis=0 for the mean to be computed along the columns of the matrix (per feature column)

train_data-=feature_mean
stdev = train_data.std(axis=0)
train_data/=stdev
# train data is now a matrix of z-scores

test_data-=feature_mean
test_data/=stdev
# notice we use same mean, and standard deviation computed on training data; never use quantities computed on testing data. not even for normalisation

In [ ]:
# model definition, and compilation
# because we'll be using k-fold validation, since the training data isn't enough to just split into a partial training set, and a validation set, we'll be partitionaling data according to k-folds cross validation, thus the model would have to be instantiated for each training partition.

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    # shape[1] corresponding to the number of features
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    # no activation used in the final layer to not restrict the range of outputs. A linear layer
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    # mse (mean squared error) for loss function, and mae (mean absolute error) for metrics
    # mse is the square of the difference between predictions and targets
    # mae is absolute value of difference between predictions and targets
    return model
# typical scalar regression model

In [ ]:
# k-folds partitioning and cross validation

k = 4 # 4-fold validation
num_validation_samples = len(train_data) // k
# print("Length of train_data: {}, num validation samples {}".format(len(train_data), num_validation_samples))

num_epochs = 100
# all_scores = []
all_mae_histories = []

for i in range(k):
    print('processing fold #: ', i)
    validation_data = train_data[i * num_validation_samples: (i+1)*num_validation_samples]
    validation_targets = train_targets[i * num_validation_samples: (i+1)*num_validation_samples]
    print("i: {} , val data & targets: [{}:{}]".format(i, i*num_validation_samples, (i+1)*num_validation_samples))

    partial_train_data = np.concatenate( [train_data[:i*num_validation_samples], train_data[(i+1)*num_validation_samples:]], axis=0 )

    partial_train_targets = np.concatenate( [train_targets[:i*num_validation_samples], train_targets[(i+1)*num_validation_samples:]], axis=0 )

    model = build_model()
    # model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    # validation_mean_squared_error, validation_mean_absolute_error = model.evaluate(validation_data, validation_targets, verbose=0)

    # all_scores.append(validation_mean_absolute_error)
    num_epochs = 500
    history = model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0, validation_data=(validation_data, validation_targets))
    # print(history.history.keys())
    mean_abs_error_history = history.history['mae']
    all_mae_histories.append(mean_abs_error_history)

In [ ]:
# building history of successive mean k-fold validation scores & plotting them
avg_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

plt.plot(range(1, len(avg_mae_history) + 1), avg_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()


In [ ]:
# Smoothing curve; omitting first 10 data points which are on a different scale than rest of graph